In [1]:
import { load } from "dotenv";
const env = await load();

In [2]:
import OpenAI from "openai";

const openai = new OpenAI({
    apiKey: env["OPENAI_API_KEY"],
    // baseURL: `https://${env["AZURE_OPENAI_API_INSTANCE_NAME"]}.openai.azure.com/openai/deployments/${env["AZURE_OPENAI_API_DEPLOYMENT_NAME"]}`,
    // defaultQuery: { 'api-version':  env["AZURE_OPENAI_API_VERSION"] },
    // defaultHeaders: { 'api-key': env["AZURE_OPENAI_API_KEY"] },
});

In [3]:
const result = await openai.chat.completions.create({
    model: 'gpt-3.5-turbo',
    // model: env["AZURE_OPENAI_API_DEPLOYMENT_NAME"],
    messages: [{ role: 'user', content: 'Say hello!' }],
  });
  console.log(result.choices[0]!.message?.content);

Hello! How can I assist you today?


In [4]:
function getCurrentWeather({ location, unit="fahrenheit"}){
   const  weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return JSON.stringify(weather_info);
}

In [5]:
const tools = [
    {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "Get the current weather in a given location",
        parameters: {
          type: "object",
          properties: {
            location: {
              type: "string",
              description: "The city and state, e.g. San Francisco, CA",
            },
            unit: { 
              type: "string", 
              enum: ["celsius", "fahrenheit"],
              description: "The unit of temperature"
            },
          },
          required: ["location", "unit"],
        },
      },
    }
]

In [6]:
 const messages = [
    {
        "role": "user",
        // "content": "北京的天气怎么样"
        "content": "What's the weather like in Redmond?"

    }
]

const result = await openai.chat.completions.create({
    model: 'gpt-3.5-turbo',
    // model: env["AZURE_OPENAI_API_DEPLOYMENT_NAME"],
    messages,
    tools
  });
  console.log(result);

{
  id: "chatcmpl-A8ij5QoqcjEFSazvxoPNxvkHwBN78",
  object: "chat.completion",
  created: 1726641915,
  model: "gpt-3.5-turbo-0125",
  choices: [
    {
      index: 0,
      message: {
        role: "assistant",
        content: null,
        tool_calls: [ [Object] ],
        refusal: null
      },
      logprobs: null,
      finish_reason: "tool_calls"
    }
  ],
  usage: {
    prompt_tokens: 88,
    completion_tokens: 20,
    total_tokens: 108,
    completion_tokens_details: { reasoning_tokens: 0 }
  },
  system_fingerprint: null
}


In [7]:
result.choices[0]

{
  index: 0,
  message: {
    role: "assistant",
    content: null,
    tool_calls: [
      {
        id: "call_a2rsNtQlQXgirlf35z78h2A0",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{"location":"Redmond","unit":"celsius"}'
        }
      }
    ],
    refusal: null
  },
  logprobs: null,
  finish_reason: "tool_calls"
}

In [8]:

const messages = [
    {
        "role": "user",
        "content": "你好"
    }
]

const result = await openai.chat.completions.create({
    model: 'gpt-3.5-turbo',
    // model: env["AZURE_OPENAI_API_DEPLOYMENT_NAME"],
    messages,
    tools,
    tool_choice: {
        type: "function",
        function: {
           name: "getCurrentWeather"
        }
    }
  });

In [9]:

const functions = {
    "getCurrentWeather": getCurrentWeather
  }

const functionInfo = result.choices[0].message.tool_calls[0].function
const functionName = functionInfo.name;
const functionParams = functionInfo.arguments

const functionResult = functions[functionName](functionParams);

console.log(functionResult);

{"temperature":"72","unit":"fahrenheit","forecast":["sunny","windy"]}


In [11]:
function getCurrentTime({ format = "iso" } = {}) {
    let currentTime;
    switch (format) {
        case "iso":
            currentTime = new Date().toISOString();
            break;
        case "locale":
            currentTime = new Date().toLocaleString();
            break;
        default:
            currentTime = new Date().toString();
            break;
    }
    return currentTime;
}

const tools = [
    {
        type: "function",
        function: {
            name: "getCurrentTime",
            description: "Get the current time in a given format",
            parameters: {
                type: "object",
                properties: {
                    format: {
                        type: "string",
                        enum: ["iso", "locale", "string"],
                        description: "The format of the time, e.g. iso, locale, string",
                    },
                },
                required: [],
            },
        },
    },
    {
        type: "function",
        function: {
          name: "getCurrentWeather",
          description: "Get the current weather in a given location",
          parameters: {
            type: "object",
            properties: {
              location: {
                type: "string",
                description: "The city and state, e.g. San Francisco, CA",
              },
              unit: { type: "string", enum: ["celsius", "fahrenheit"] },
            },
            required: ["location", "unit"],
          },
        },
      }
];

In [12]:

const messages = [
    { role: "user", content: " 上海 新疆 这三个城市的天气如何?" },
]

const result = await openai.chat.completions.create({
    model: 'gpt-3.5-turbo',
    // model: env["AZURE_OPENAI_API_DEPLOYMENT_NAME"],
    messages,
    tools
  });


In [13]:
result.choices[0]

{
  index: 0,
  message: {
    role: "assistant",
    content: null,
    tool_calls: [
      {
        id: "call_9vNp68f5fBVNEqvWgs1FiuHP",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{"location": "Shanghai", "unit": "celsius"}'
        }
      },
      {
        id: "call_lCdUC4yNVyWLwOFKMS5LHZXZ",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{"location": "Xinjiang", "unit": "celsius"}'
        }
      }
    ],
    refusal: null
  },
  logprobs: null,
  finish_reason: "tool_calls"
}

In [14]:

messages.push(result.choices[0].message)

const functions = {
    "getCurrentWeather": getCurrentWeather
  }

const cell = result.choices[0].message.tool_calls[0]
const functionInfo = cell.function
const functionName = functionInfo.name;
const functionParams = functionInfo.arguments
const functionResult = functions[functionName](functionParams);

console.log(messages);
messages.push({
  tool_call_id: cell.id,
  role: "tool",
  name: functionName,
  content: functionResult,
}); 

const response = await openai.chat.completions.create({
  model: 'gpt-3.5-turbo',
  // model: env["AZURE_OPENAI_API_DEPLOYMENT_NAME"],
  messages,
});
console.log(response);

[
  { role: "user", content: " 上海 新疆 这三个城市的天气如何?" },
  {
    role: "assistant",
    content: null,
    tool_calls: [
      {
        id: "call_9vNp68f5fBVNEqvWgs1FiuHP",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{"location": "Shanghai", "unit": "celsius"}'
        }
      },
      {
        id: "call_lCdUC4yNVyWLwOFKMS5LHZXZ",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{"location": "Xinjiang", "unit": "celsius"}'
        }
      }
    ],
    refusal: null
  }
]


Error: 400 An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_lCdUC4yNVyWLwOFKMS5LHZXZ

In [ ]:
response.choices[0].message

In [ ]:
messages